In [1]:
import pandas as pd
import numpy as np
import math
import re

In [2]:
df_crafts = pd.read_csv('crafts.csv', sep='\t', encoding='utf-8')
display(df_crafts.tail())

,Unnamed: 0,attr,charge,id,mode,name,race,rare,skill1,skill2,skill3
299,299,NaN,連擊 (Combo) 達 4 或以上,300,3,連之幻變龍咒,NaN,3,1 回合內，迴避火屬性敵人的攻擊,完成龍刻脈動後，將場上數量最多的 1 種符石轉化為水符石,隨機 3 個水成員的技能 CD 減少 2
300,300,NaN,連擊 (Combo) 達 4 或以上,301,2,連之幻變龍印,NaN,3,1 回合內，迴避木屬性敵人的攻擊,完成龍刻脈動後，將場上數量最多的 1 種符石轉化為火符石,隨機 3 個火成員的技能 CD 減少 2
301,301,NaN,連擊 (Combo) 達 4 或以上,302,4,連之幻變龍符,NaN,3,1 回合內，迴避光屬性敵人的攻擊,完成龍刻脈動後，將場上數量最多的 1 種符石轉化為暗符石,隨機 3 個暗成員的技能 CD 減少 2
302,302,NaN,連擊 (Combo) 達 4 或以上,303,6,連之靈巧龍刃,NaN,3,1 回合內，迴避暗屬性敵人的攻擊,完成龍刻脈動後，將場上數量最多的 1 種符石轉化為光符石,隨機 3 個光成員的技能 CD 減少 2
303,303,NaN,連擊 (Combo) 達 4 或以上,304,9,連之邃矚龍丸,NaN,3,1 回合內，所有成員無視屬性相剋,1 回合內，隊伍攻擊力提升 20%,1 回合內，完全回復生命力


## Translate string to value

In [3]:
def Cno_format(no):
    if no < 10:
        str_no = str(no).zfill(2)
    else:
        str_no = str(no)
    return str_no

In [4]:
def series_to_string(craft):
    for i, v in craft.iteritems():
        if isinstance(v, str) is False:
            if math.isnan(v):
                craft[i] = "0"
            else:
                craft[i] = str(craft[i])
    craft["attr"] = attr_to_value(craft["attr"])
    craft["race"] = race_to_value(craft["race"])
    craft["charge"] = charge_to_value(craft["charge"])
    craft["skilltype"] = find_skill_types(craft)
    return craft

In [5]:
def attr_to_value(s):
    attr = ["水", "火", "木", "光", "暗"]
    str2 = 0
    for i in range(5):
        if s == attr[i]:
            str2 = i+1
    str2 = str(str2)
    return str2

In [6]:
def race_to_value(s):
    race = ["人類", "獸類", "神族", "龍類", "妖精類", "魔族", "機械族"]
    str2 = 0
    for i in range(7):
        if s == race[i]:
            str2 = i+1
    str2 = str(str2)
    return str2

In [7]:
def charge_to_value(s):
    charge = ["任何符石", "水符石", "火符石", "木符石", "光符石", "暗符石", "心符石", "發動攻擊", "受到攻擊", "連擊"]
    str2 = 0
    for i in range(10):
        result = re.search(charge[i], s)
        if result is not None:
            str2 = i+1
    str2 = str(str2)
    return str2

## Classifying skill type

In [8]:
def find_skill_type(s):
    skill_set = set()
    skill_type = ['攻擊力(.*)提升', '回復力(.*)提升', '回復(.*)生命力','轉化為我方生命力',
                  '轉化(.*)符石', '屬性相剋', 'CD', 
                  '減少所受傷害', '迴避', '所受傷害不會使你死亡', '延遲',
                  '造成(.*)', '造成(.*)此傷害無視防禦力',
                  '無視(.*)指定形狀盾', '無視(.*)五屬盾', '無視(.*)攻前盾',
                  '防禦力下降', '無視全體敵人(.*)防禦力', '追打', '反擊', '單體攻擊轉化為全體攻擊']
    for i in range(len(skill_type)):
        skill = skill_type[i]
        pattern_detail = r'(.*)' + skill + r'(.*)'
        s2 = re.search(pattern_detail, s)
        if s2 is not None:
            skill_set.add(str(i+1))
    return skill_set

In [9]:
def find_skill_types(craft):
    skill_set = set()
    skill_str = craft['skill1']
    skill_set |= find_skill_type(skill_str)
    skill_str = craft['skill2']
    if isinstance(skill_str, str) is True:
        skill_set |= find_skill_type(skill_str)
    skill_str = craft['skill3']
    if isinstance(skill_str, str) is True:
        skill_set |= find_skill_type(skill_str)
    skilltype_str = " ".join(skill_set)
    return skilltype_str

In [10]:
def data_to_wikitext(craft):
    str_Cno = Cno_format(int(craft["id"]))
    str_craft = "<div data-id=\"" + craft["id"] + "\"" + " data-star=\"" + craft["rare"] + "\"" \
    + " data-attribute=\"" + craft["attr"] + "\"" + " data-racialtype=\"" + craft["race"] + "\"" \
    + " data-mode=\"" + craft["mode"] + "\"" + " data-charge=\""  + craft["charge"] + "\"" \
    + " data-skilltype=\"" + craft["skilltype"] + "\"" \
    + " data-skilltype2=\"" + craft["skilltype"] + "\"" \
    + ">{{C" + str_Cno + "|60}}" + "</div>\n"
    return str_craft

In [11]:
str_all =""
for index, row in df_crafts.iterrows():
    row2 = series_to_string(row)
    str_craft = data_to_wikitext(row2)
    print(str_craft)
    str_all += str_craft
# print(str_all)


<div data-id="1" data-star="1" data-attribute="0" data-racialtype="0" data-mode="1" data-charge="1" data-skilltype="1" data-skilltype2="1">{{C01|60}}</div>

<div data-id="2" data-star="1" data-attribute="0" data-racialtype="0" data-mode="2" data-charge="1" data-skilltype="1" data-skilltype2="1">{{C02|60}}</div>

<div data-id="3" data-star="1" data-attribute="0" data-racialtype="0" data-mode="3" data-charge="1" data-skilltype="1" data-skilltype2="1">{{C03|60}}</div>

<div data-id="4" data-star="2" data-attribute="1" data-racialtype="0" data-mode="1" data-charge="2" data-skilltype="3 1" data-skilltype2="3 1">{{C04|60}}</div>

<div data-id="5" data-star="2" data-attribute="2" data-racialtype="0" data-mode="3" data-charge="3" data-skilltype="3 1" data-skilltype2="3 1">{{C05|60}}</div>

<div data-id="6" data-star="2" data-attribute="3" data-racialtype="0" data-mode="1" data-charge="4" data-skilltype="3 1" data-skilltype2="3 1">{{C06|60}}</div>

<div data-id="7" data-star="2" data-attribute=


<div data-id="89" data-star="2" data-attribute="5" data-racialtype="0" data-mode="1" data-charge="10" data-skilltype="3 17" data-skilltype2="3 17">{{C89|60}}</div>

<div data-id="90" data-star="3" data-attribute="0" data-racialtype="4" data-mode="2" data-charge="7" data-skilltype="3 7 1" data-skilltype2="3 7 1">{{C90|60}}</div>

<div data-id="91" data-star="3" data-attribute="0" data-racialtype="1" data-mode="3" data-charge="10" data-skilltype="12 6 3 1" data-skilltype2="12 6 3 1">{{C91|60}}</div>

<div data-id="92" data-star="3" data-attribute="2" data-racialtype="3" data-mode="1" data-charge="1" data-skilltype="3 7 1" data-skilltype2="3 7 1">{{C92|60}}</div>

<div data-id="93" data-star="3" data-attribute="0" data-racialtype="6" data-mode="1" data-charge="10" data-skilltype="10 1" data-skilltype2="10 1">{{C93|60}}</div>

<div data-id="94" data-star="3" data-attribute="0" data-racialtype="0" data-mode="3" data-charge="10" data-skilltype="2 1 8" data-skilltype2="2 1 8">{{C94|60}}</div

<div data-id="178" data-star="3" data-attribute="0" data-racialtype="6" data-mode="8" data-charge="10" data-skilltype="16 7 1" data-skilltype2="16 7 1">{{C178|60}}</div>

<div data-id="179" data-star="1" data-attribute="0" data-racialtype="0" data-mode="6" data-charge="8" data-skilltype="2 1" data-skilltype2="2 1">{{C179|60}}</div>

<div data-id="180" data-star="1" data-attribute="0" data-racialtype="0" data-mode="6" data-charge="10" data-skilltype="1" data-skilltype2="1">{{C180|60}}</div>

<div data-id="181" data-star="2" data-attribute="1" data-racialtype="0" data-mode="6" data-charge="2" data-skilltype="7 1" data-skilltype2="7 1">{{C181|60}}</div>

<div data-id="182" data-star="2" data-attribute="2" data-racialtype="0" data-mode="6" data-charge="3" data-skilltype="7 1" data-skilltype2="7 1">{{C182|60}}</div>

<div data-id="183" data-star="2" data-attribute="3" data-racialtype="0" data-mode="6" data-charge="4" data-skilltype="7 1" data-skilltype2="7 1">{{C183|60}}</div>

<div data-id

<div data-id="276" data-star="3" data-attribute="3" data-racialtype="3" data-mode="1" data-charge="4" data-skilltype="10 7 1" data-skilltype2="10 7 1">{{C276|60}}</div>

<div data-id="277" data-star="3" data-attribute="3" data-racialtype="1" data-mode="2" data-charge="4" data-skilltype="6 1 4" data-skilltype2="6 1 4">{{C277|60}}</div>

<div data-id="278" data-star="3" data-attribute="3" data-racialtype="5" data-mode="4" data-charge="4" data-skilltype="7 5 1" data-skilltype2="7 5 1">{{C278|60}}</div>

<div data-id="279" data-star="3" data-attribute="4" data-racialtype="4" data-mode="1" data-charge="5" data-skilltype="7 5 1" data-skilltype2="7 5 1">{{C279|60}}</div>

<div data-id="280" data-star="3" data-attribute="4" data-racialtype="2" data-mode="3" data-charge="5" data-skilltype="3 1 18" data-skilltype2="3 1 18">{{C280|60}}</div>

<div data-id="281" data-star="3" data-attribute="4" data-racialtype="6" data-mode="7" data-charge="5" data-skilltype="16 5 1" data-skilltype2="16 5 1">{{C28

In [12]:
text_file = open("Output_to_wiki.txt", "w")
text_file.write(str_all)
text_file.close()